In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv("https://github.com/SavioSal/datasets/raw/master/movie_clean.csv", encoding='unicode_escape')

In [3]:
movies.head()

,title,genres,runtime,vote_average,vote_count,year,overview
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995.0,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995.0,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995.0,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995.0,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995.0,Just when George Banks has recovered from his ...


In [4]:
movies_new = movies[movies['vote_average']>7]
movies_new.shape

(7714, 7)

In [5]:
movies_new = movies_new.reset_index()

In [6]:
movies_new.tail()

,index,title,genres,runtime,vote_average,vote_count,year,overview
7709,45418,Dyketactics,['drama'],4.0,8.0,1.0,1974.0,Klasa II B jako zadanie domowe otrzymuje do na...
7710,45421,Pickpocket,"['drama', 'foreign']",105.0,7.2,13.0,1997.0,"It's Christmas, and children at an orphanage i..."
7711,45437,In a Heartbeat,"['family', 'animation', 'romance', 'comedy']",4.0,8.3,146.0,2017.0,Almaâs family has been producing quality oli...
7712,45443,The Man with the Rubber Head,"['comedy', 'fantasy', 'science fiction']",3.0,7.6,29.0,1901.0,"In the future, national boundaries have been b..."
7713,45462,Century of Birthing,['drama'],360.0,9.0,3.0,2011.0,Lt. Bobby Quinn is a Special Forces commando k...


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
movies_new['overview'].isna().sum()

156

In [9]:
movies_new['overview'].fillna("", inplace=True)

In [10]:
movies_new['overview'].isna().sum()

0

In [11]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_new['overview'])

In [12]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [13]:
cosine_sim.shape

(7714, 7714)

In [14]:
cosine_sim[0]

array([1.        , 0.        , 0.        , ..., 0.        , 0.00434795,
       0.        ])

In [15]:
indices = pd.Series(movies_new.index, index = movies_new['title']).drop_duplicates()
indices.head()

title
Toy Story    0
Heat         1
Balto        2
Nixon        3
Casino       4
dtype: int64

In [16]:
similarity_scores = list(enumerate(cosine_sim[0]))
similarity_scores = sorted(similarity_scores, key = lambda x:x[1], reverse = True)
similarity_scores[1:11]

[(3029, 0.5177099048466635),
 (7267, 0.21184972709165517),
 (235, 0.1666072877377134),
 (3371, 0.1387599360974652),
 (2384, 0.1360853436429296),
 (5081, 0.11057769270196234),
 (753, 0.10571923427313008),
 (3993, 0.09946695661989194),
 (474, 0.0892602578779177),
 (3230, 0.08460592218131369)]

In [17]:
def movie_recommendations(title):
  idx = indices[title]
  similarity_scores = list(enumerate(cosine_sim[idx]))
  similarity_scores = sorted(similarity_scores, key = lambda x:x[1], reverse = True)
  final_scores = similarity_scores[1:11]
  movie_indices = [i[0] for i in final_scores]
  return movies_new['title'].iloc[movie_indices]

In [18]:
movies_new.head(5)

,index,title,genres,runtime,vote_average,vote_count,year,overview
0,0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995.0,"Led by Woody, Andy's toys live happily in his ..."
1,5,Heat,"['action', 'crime', 'drama', 'thriller']",170.0,7.7,1886.0,1995.0,"Obsessive master thief, Neil McCauley leads a ..."
2,12,Balto,"['family', 'animation', 'adventure']",78.0,7.1,423.0,1995.0,An outcast half-wolf risks his life to prevent...
3,13,Nixon,"['history', 'drama']",192.0,7.1,72.0,1995.0,An all-star cast powers this epic look at Amer...
4,15,Casino,"['drama', 'crime']",178.0,7.8,1343.0,1995.0,The life of the gambling paradise â Las Vega...


In [19]:
watched = input("enter the movie you watched")
movie_recommendations(watched)

enter the movie you watched Heat


3450                         Woyzeck
4038    The Woman in the Septic Tank
7589                        Radiator
5066                       Furious 7
1882              Days of Being Wild
6770         Maheshinte Prathikaaram
3921            Last Cowboy Standing
2091                Ivan's Childhood
6829                  Audrie & Daisy
5436                         Mustang
Name: title, dtype: object

In [20]:
# More search tools here: https://colab.research.google.com/drive/1wOlSiLYfUJEOKu1mor9Tlfz38Ucof6C5#scrollTo=1fZ0yduaXjNJ